In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
import cv2 

In [48]:
trainingDataPath = r"C:\Users\theha\Downloads\archive\train"
testDataPath = r"C:\Users\theha\Downloads\archive\test"
categories = ["angry","happy","neutral","sad"]

In [49]:
x_train = []
y_train = []
img_size = 224
def create_training_data():
  for category in categories:
    path = os.path.join(trainingDataPath, category)
    class_num = categories.index(category)
    for file in os.listdir(path):
      try:
        img = cv2.imread(os.path.join(path, file))
        resized_img = cv2.resize(img, (img_size, img_size))
        x_train.append(resized_img)
        y_train.append(class_num)
      except Exception as e:
        pass
    
x_test = []
y_test = []
def create_testing_data():
  for category in categories:
    path = os.path.join(testDataPath, category)
    class_num = categories.index(category)
    for file in os.listdir(path):
      try:
        img = cv2.imread(os.path.join(path, file))
        resized_img = cv2.resize(img, (img_size, img_size))
        x_test.append(resized_img)
        y_test.append(class_num)
      except Exception as e:
        pass

In [50]:
create_training_data()
x_train = np.array(x_train).reshape(-1, img_size, img_size, 3)
y_train = np.array(y_train)

create_testing_data()
x_test = np.array(x_test).reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)

In [38]:
# normalization_layer = layers.Rescaling(1./255)

# normalized_x_train = [normalization_layer(x) for x in x_train]
# normalized_x_test = [normalization_layer(x) for x in x_test]
# image_batch, labels_batch = next(iter(normalized__x_train))

In [51]:
num_classes = len(categories)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_size, img_size, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [52]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [53]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_7 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 224, 224, 16)      448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 112, 112, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 112, 112, 32)      4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 56, 56, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 56, 56, 64)       

In [ ]:
epochs=10
history= model.fit(
  x_train,
    y_train,
  epochs=epochs,
    batch_size=32,
    validation_data=(x_test, y_test),
)

Epoch 1/10
657/657 [==============================] - 249s 379ms/step - loss: 1.0989 - accuracy: 0.5246 - val_loss: 1.0608 - val_accuracy: 0.5368
Epoch 2/10
657/657 [==============================] - 240s 365ms/step - loss: 0.9764 - accuracy: 0.5854 - val_loss: 1.0405 - val_accuracy: 0.5547
Epoch 3/10
657/657 [==============================] - 238s 363ms/step - loss: 0.8569 - accuracy: 0.6453 - val_loss: 1.0231 - val_accuracy: 0.5568
Epoch 4/10
657/657 [==============================] - 237s 361ms/step - loss: 0.6975 - accuracy: 0.7194 - val_loss: 1.1598 - val_accuracy: 0.5512
Epoch 5/10
657/657 [==============================] - 238s 363ms/step - loss: 0.5076 - accuracy: 0.8020 - val_loss: 1.2721 - val_accuracy: 0.5566
Epoch 6/10
285/657 [============>.................] - ETA: 2:09 - loss: 0.3013 - accuracy: 0.8945

In [57]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(x_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
41/41 [==============================] - 14s 327ms/step - loss: 1.1633 - accuracy: 0.4937
test loss, test acc: [1.1633094549179077, 0.49366846680641174]
Generate predictions for 3 samples
1/1 [==============================] - 0s 68ms/step
predictions shape: (3, 4)


In [43]:
model.save(r"second_model/")

INFO:tensorflow:Assets written to: first_model/assets


INFO:tensorflow:Assets written to: first_model/assets


In [45]:
new_model = keras.models.load_model(r"first_model/")

In [46]:
print("Evaluate on test data")
results = new_model.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
57/57 [==============================] - 19s 326ms/step - loss: 4.4102 - accuracy: 0.4926
test loss, test acc: [4.410184383392334, 0.49261632561683655]


In [47]:
model.save_weights(r"second_weights/")